In [1]:
import random
import numpy as np
import pandas as pd
from itertools import product
import multiprocessing as mp
import time
import mdptoolbox.example

In [42]:
class ValueIteration:
    def __init__(self, Q=5, Tp=3, Tnp=3, p=0.5, P1=0.3, P2=0.5, W1=5, W2=3, arrival_rate=0.2):
        self.Q = Q
        self.Tp = Tp ## priority time
        self.Tnp = Tnp ## non-priority time
        self.W1 = W1 ## cost of priority
        self.W2 = W2 ## cost of non-priority
        self.arrival_rate = arrival_rate 
        self.p = p
        self.P1 = (1-np.exp(-Tp*arrival_rate))
        self.P2 = (1-np.exp(-Tnp*arrival_rate))
        if arrival_rate * Tp > 1 or arrival_rate * Tnp > 1:
            print('Arrival rate too high')
            raise AssertionError("Arrival rate too high")
        
        self.action_times = {
            "0" : 1 / arrival_rate,
            "1" : Tp,
            "2" : Tnp
        }
        self.states = [(i, j) for i in range(self.Q + 1) for j in range(self.Q - i + 1)]
            
    def sample_next_state(self, state, action):
        priority_state = state[0]
        non_priority_state = state[1]
        ## Hold state
        if action == 0:
            if (priority_state + non_priority_state) == self.Q:
                cost = 1e12
                prob = 1
                arrival_time = (1 / self.arrival_rate)
            else:
                arrival_time = 1 / self.arrival_rate
                cost = (self.W1*priority_state + self.W2*non_priority_state) * (1 / self.arrival_rate)
                if random.random() < self.p:
                    # priority arrival
                    priority_state += 1
                    prob = self.p
                else:
                    non_priority_state += 1
                    prob = 1- self.p
            arrival_time = (1 / self.arrival_rate)
                
        if action == 1:
            if (priority_state == 0):
                cost = 1e12
                prob = 1
                arrival_time = self.Tp
            else:
                arrival_time = np.random.exponential(1 / self.arrival_rate)
                arrival_time = arrival_time if arrival_time < self.Tp else self.Tp
                priority_state -= 1
                arrival_time = self.Tp
                if random.random() < self.P1:
                    # Probability of Arrivals
                    prob = self.P1
                    if random.random() < self.p:
                        # priority arrival
                        priority_state += 1
                        cost =(self.W1*priority_state + self.W2*non_priority_state) * self.Tp+ (self.W1 * (self.Tp- ((1-np.exp(-self.arrival_rate*self.Tp))/(self.arrival_rate))))
                        prob *= self.p
                    else:
                        # non priority arrival
                        non_priority_state += 1
                        cost = (self.W1*priority_state + self.W2*non_priority_state) * self.Tp+(self.W2 * (self.Tp-((1-np.exp(-self.arrival_rate*self.Tp))/(self.arrival_rate))))
                        prob *= (1-self.p)
                else:
                    # No arrival
                    prob = 1 - self.P1
                    cost =(self.W1*priority_state + self.W2*non_priority_state) * self.Tp
            arrival_time=self.Tp
                    
        if action == 2:
            if (non_priority_state == 0):
                cost = 1e12
                prob = 1
                arrival_time = 0
            else:
                arrival_time = np.random.exponential(1 / self.arrival_rate)
                arrival_time = arrival_time if arrival_time < self.Tnp else self.Tnp
                non_priority_state -= 1
                arrival_time = self.Tnp
                if random.random() < self.P2:
                    # Probability of Arrivals
                    prob = self.P2
                    if random.random() < self.p:
                        # priority arrival
                        priority_state += 1
                        cost =(self.W1*priority_state + self.W2*non_priority_state) * self.Tnp + (self.W1 * (self.Tnp-((1-np.exp(-self.arrival_rate*self.Tnp))/(self.arrival_rate))))
                        prob *= self.p
                    else:
                        # non priority arrival
                        non_priority_state += 1
                        cost = (self.W1*priority_state + self.W2*non_priority_state) * self.Tnp+(self.W2 * (self.Tnp-((1-np.exp(-self.arrival_rate*self.Tnp))/(self.arrival_rate))))
                        prob *= (1-self.p)
                else:
                    # No arrival
                    prob = 1 - self.P2
                    cost = (self.W1*priority_state + self.W2*non_priority_state) * self.Tnp
            arrival_time=self.Tnp
                    
#         return non_priority_state, cost, priority_state, prob, arrival_time 
        return priority_state, non_priority_state, cost, prob,arrival_time
                
    
    def get_transition_set(self, state, action):
        tset = set()
        values ={}
        count = {}
        for _ in range(1000):
#             non_priority, cost, params, prob, time_for_discounting = self.sample_next_state(state, action)
            priority_state, non_priority_state, cost, prob,arrival_time = self.sample_next_state(state, action)
            next_state = (priority_state, non_priority_state)
#             if values.get(next_state) != None:
#                 count[next_state] = count[next_state] + 1
#                 values[next_state] = values[next_state] + (cost - values[next_state])/count[next_state]
#             else:
#                 count[next_state] = 1
#                 values[next_state] = cost
            tset.add((next_state, prob,cost,arrival_time))
#         print(tset)
#         output = self.sample_next_state(state,action)
#         print(output)
#         print("state:",state,"action:",action)
#         print("next_state_with_cost:",values)
        return tset
    
    def action_0(self, state):
        if state[0] + state[1] == self.Q:
            return [(state, 1, 1e12)]
        cost = (self.W1 * state[0] + self.W2 * state[1])  / self.arrival_rate
        transitions = [(((state[0] + 1, state[1])), self.p, cost), ((state[0], state[1] + 1), 1- self.p, cost)]
        
        return transitions
    
    def action_1(self, state):
        if state[0] == 0:
            return [(state , 1, 1e12)]
        
        transitions = []
        # N.A
        cost = ((state[0] - 1)* self.W1 + state[1] * self.W2) * self.Tp
        transitions.append(((state[0] - 1, state[1]), 1 - self.P1, cost))

        # P.A
        priority_arrival_cost = self.W1 * (self.Tp- ((1-np.exp(-self.arrival_rate*self.Tp))/(self.arrival_rate)))
        transitions.append((state, self.p * self.P1, cost + priority_arrival_cost))

        # N.P.A
        non_priority_cost = self.W2 * (self.Tp-((1-np.exp(-self.arrival_rate*self.Tp))/(self.arrival_rate)))
        transitions.append(((state[0]-1, state[1] + 1), (1 - self.p) * self.P1, cost + non_priority_cost))
        return transitions
    
    def action_2(self, state):
        if state[1] == 0:
            return [(state, 1, 1e12)]
        transitions = []

        # N.A
        cost = (state[0] * self.W1 + (state[1] - 1) * self.W2) * self.Tnp
        transitions.append(((state[0], state[1] - 1), 1 - self.P2, cost))

        # P.A
        priority_arrival_cost = (self.W1 * (self.Tnp-((1-np.exp(-self.arrival_rate*self.Tnp))/(self.arrival_rate))))
        transitions.append(((state[0] + 1, state[1]-1), self.p * self.P2, cost + priority_arrival_cost))

        # N.P.A
        non_priority_cost = (self.W2 * (self.Tnp-((1-np.exp(-self.arrival_rate*self.Tnp))/(self.arrival_rate))))
        transitions.append(((state[0], state[1]), (1 - self.p) * self.P2, cost + non_priority_cost))

        return transitions
    
    def get_transition_reward_matrix(self):        

        transition_reward_matrix = {}

        for state in self.states:
            action_0_result = self.action_0(state)
            action_1_result = self.action_1(state)
            action_2_result = self.action_2(state)
            transition_reward_matrix[state] = (action_0_result, action_1_result, action_2_result)
            transition_reward_matrix[state] = {
                0 : action_0_result,
                1 : action_1_result,
                2 : action_2_result
            }
        return transition_reward_matrix

    
    def value_iteration(self, gamma=0.01, theta=1e-3, max_iterations=10000):
        states = [[i, j] for i in range(self.Q + 1) for j in range(self.Q - i + 1)]
        transition_reward_matrix = {}
        actions = [0, 1, 2]

        for state in states:
            transition_reward_matrix[tuple(state)] = {}
#             print(state)
            for action in actions:
                tset = self.get_transition_set(state, action)
                transition_reward_matrix[tuple(state)][action] = tset
        
#         print(transition_reward_matrix)
#         return
        value_function = {tuple(state): 0 for state in transition_reward_matrix}
        Q_values = {tuple(state): {action: 0 for action in actions} for state in transition_reward_matrix}
        policy = {}

        delta = float('inf')
        iteration = 0
        while delta > theta and iteration < max_iterations:
            iteration += 1
            print(f"Iteration: {iteration}")
            delta = 0
            for state, action_dict in transition_reward_matrix.items():
                old_value = value_function[state]
                action_values = []
                for action, tset in action_dict.items():
                    action_value = 0 
                    for transition in tset:
                        next_state = transition[0]
                        prob = transition[1]
                        cost = transition[2]
                        time_for_discounting = transition[3]
                        print("Here you go:state,action, next_state,prob,cost,time_for_discounting",state,action, next_state,prob,cost,time_for_discounting)
                        # print(state, next_state, prob)
                        # time.sleep(10)
#                         dummy_1, dummy_2, cost, dummy_3,time_for_discounting=self.sample_next_state(state, action)
#                         next_state = (dummy_1,dummy_2)
#                         prob = dummy_3
#                         action_value += ( 1/(1+cost) + np.exp(-time_for_discounting*gamma) * value_function[next_state])*prob
#                         action_value += ( 1/(1+cost) + 0.011* value_function[next_state])*prob
#                         action_value += ( 1/(1+cost) + 0.11* value_function[next_state])*prob
#                        action_value += (-1 * cost + 0.9 * value_function[next_state])*prob
                        action_value += (-cost + np.exp(-time_for_discounting*gamma) * value_function[next_state])*prob
                        
                    Q_values[state][action] = action_value
                    action_values.append(action_value)

                value_function[state] = np.max(action_values)
                policy[state] = np.argmax(action_values)
                delta = max(delta, abs(value_function[state] - old_value))
                print(f"State: {state}, Value: {value_function[state]}, Policy: {policy[state]}",)
            
#             for item in value_function:
#                 value_function[item] -= value_function[(0,0)]

            
        return value_function, Q_values, policy, transition_reward_matrix
    
    
    def export_transition_reward_matrix(self, transition_reward_matrix):
        transition_list = []
        reward_list = []
        for state, action_dict in transition_reward_matrix.items():
            for action, tset in action_dict.items():
                for transition in tset:
                    next_state, prob = transition
                    transition_list.append((state, action, next_state, prob))
                    # reward_list.append((state, action, next_state, cost))

        transition_df = pd.DataFrame(transition_list, columns=["state", "action", "next_state", "probability"])
        # reward_df = pd.DataFrame(reward_list, columns=["state", "action", "next_state", "reward"])

        transition_df.to_csv("transition_matrix.csv", index=False)
        # reward_df.to_csv("reward_matrix.csv", index=False)

    def run_simulation_for_params(self,params):
        """
        Run simulation for a given set of parameters and return the results.
        """
        vi_instance = ValueIteration(**params)
        _, Q_values, _, transition_reward_matrix = vi_instance.value_iteration()

        results = []
        for state, actions in Q_values.items():
            for action, Q_value in actions.items():
                result_row = {**params, 'state': state, 'action': action, 'Q_value': Q_value}
                results.append(result_row)

        # Export transition and reward matrices for this set of parameters
        self.export_transition_reward_matrix(transition_reward_matrix)

        return results
    
    
    def run_simulations(self, parameter_values):
        param_names = list(parameter_values.keys())
        param_combinations = list(product(*parameter_values.values()))

        # Prepare the pool for multiprocessing
        pool = mp.Pool(20)
 
        # Create a list of dictionaries of parameters
        param_dicts = [dict(zip(param_names, combination)) for combination in param_combinations]

        # Step 1: Run simulations in parallel
        all_results = pool.map(self.run_simulation_for_params, param_dicts)

        # Step 2: Flatten the list of results
        flat_results = [item for sublist in all_results for item in sublist]

        # Step 3: Create DataFrame from results list
        result_df = pd.DataFrame(flat_results)

        # Save to CSV with all parameters as separate columns
        result_df.to_csv("Q_values_simulations.csv", index=False)

        pool.close()
        pool.join()

In [ ]:
# asd = ValueIteration(5,3,3,0.5,0.3,0.5,405,3,0.2)

# value_function, Q_values, _, transition_reward_matrix = asd.value_iteration()
# # transition_reward_matrix.to_csv('transition_reward_matrix.csv')

# import pandas as pd
# dataframe = pd.DataFrame(transition_reward_matrix)
# dataframe.to_csv("data.csv", header=True)

# # results = []
# # for state, actions in Q_values.items():
# #     for action, Q_value in actions.items():
# #         result_row = {**params, 'state': state, 'action': action, 'Q_value': Q_value}
# #         results.append(result_row)


In [54]:
Q = 5
asd = ValueIteration(Q,3,3,0.5,0.3,0.5,35, 1,0.2)
states = np.array([[i, j] for i in range(Q + 1) for j in range(Q - i + 1)])
P1 = np.zeros((3,len(states),len(states)))
R1 = np.zeros((3,len(states),len(states)))
for actions in [0,1,2]:
    for i in range(len(states)):
        transition_set = asd.get_transition_set(states[i],actions)
        for next_state,prob,cost,arrival_time in transition_set:
            P1[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = prob
            R1[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = -cost

In [55]:
Q = 5
asd = ValueIteration(Q,3,3,0.5,0.3,0.5,35, 1,0.2)
transition_reward_matrix = asd.get_transition_reward_matrix()

states = np.array([[i, j] for i in range(Q + 1) for j in range(Q - i + 1)])
P2 = np.zeros((3,len(states),len(states)))
R2 = np.zeros((3,len(states),len(states)))
for actions in [0,1,2]:
    for i in range(len(states)):
        transition_set = transition_reward_matrix[tuple(states[i])][actions]
        for next_state,prob,cost in transition_set:
            P2[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = prob
            R2[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = -cost

In [56]:
# P, R = mdptoolbox.example.forest()
vi = mdptoolbox.mdp.ValueIteration(P1, R1,0.8)
vi.run()
result = vi.policy # result is (0, 0, 0)import mdptoolbox.example
for i in range(len(states)):
    print(f"States:{states[i]}, Values:{vi.V[i]}, Policy:{result[i]}")

States:[0 0], Values:-82.17621120249632, Policy:0
States:[0 1], Values:-102.59525507023973, Policy:0
States:[0 2], Values:-122.22189963024346, Policy:0
States:[0 3], Values:-140.57218005865835, Policy:2
States:[0 4], Values:-157.87394052061387, Policy:2
States:[0 5], Values:-174.45532861025714, Policy:2
States:[1 0], Values:-103.43412619537075, Policy:1
States:[1 1], Values:-122.35577109531951, Policy:1
States:[1 2], Values:-140.57218005865835, Policy:1
States:[1 3], Values:-157.87394052061387, Policy:1
States:[1 4], Values:-174.45532861025714, Policy:1
States:[2 0], Values:-247.2975907901182, Policy:1
States:[2 1], Values:-265.10635876848886, Policy:1
States:[2 2], Values:-282.3363339918648, Policy:1
States:[2 3], Values:-298.91772208150223, Policy:1
States:[3 0], Values:-483.9301747092224, Policy:1
States:[3 1], Values:-500.9253747523716, Policy:1
States:[3 2], Values:-517.4682200269737, Policy:1
States:[4 0], Values:-790.7580897126643, Policy:1
States:[4 1], Values:-807.165952524803

In [57]:
# P, R = mdptoolbox.example.forest()
vi = mdptoolbox.mdp.ValueIteration(P2, R2,0.8)
vi.run()
result = vi.policy # result is (0, 0, 0)import mdptoolbox.example
for i in range(len(states)):
    print(f"States:{states[i]}, Values:{vi.V[i]}, Policy:{result[i]}")

States:[0 0], Values:-16.235671756961818, Policy:0
States:[0 1], Values:-20.43481821877366, Policy:2
States:[0 2], Values:-28.014258859647782, Policy:2
States:[0 3], Values:-37.91605281465993, Policy:2
States:[0 4], Values:-49.413252034233054, Policy:2
States:[0 5], Values:-62.00581838373112, Policy:2
States:[1 0], Values:-20.43481821877366, Policy:1
States:[1 1], Values:-28.014258859647782, Policy:1
States:[1 2], Values:-37.91605281465993, Policy:1
States:[1 3], Values:-49.413252034233054, Policy:1
States:[1 4], Values:-62.00581838373112, Policy:1
States:[2 0], Values:-152.47665233075298, Policy:1
States:[2 1], Values:-162.3784462857473, Policy:1
States:[2 2], Values:-173.87564550532042, Policy:1
States:[2 3], Values:-186.4682118548185, Policy:1
States:[3 0], Values:-380.9289442226703, Policy:1
States:[3 1], Values:-392.426143441637, Policy:1
States:[3 2], Values:-405.01870979113505, Policy:1
States:[4 0], Values:-682.1031168476466, Policy:1
States:[4 1], Values:-694.6956831875552, Po

In [58]:
rvi = mdptoolbox.mdp.RelativeValueIteration(P1,R1)
rvi.setVerbose()

rvi.run()
result = rvi.policy # result is (0, 0, 0)import mdptoolbox.example
for i in range(len(states)):
    print(f"States:{states[i]}, Values:{rvi.V[i]}, Policy:{result[i]}")

  Iteration		U variation
    1		  450.4069587266717
    2		  361.7835264882077
    3		  274.28976040215565
    4		  192.33581337876996
    5		  110.31650960454039
    6		  49.621511725821364
    7		  20.107764814885172
    8		  8.15916423431355
    9		  6.413079487635059
    10		  4.8238807846678355
    11		  3.454153415699011
    12		  2.34130647191364
    13		  1.5271782625022752
    14		  0.9578017076532888
    15		  0.5845741193425056
    16		  0.34683723544230816
    17		  0.2018324189480154
    18		  0.1149935439834735
    19		  0.06460020816712131
    20		  0.0356977844191988
    21		  0.01952200044820529
    22		  0.010534538472029453
    23		  0.005640823033445486
Iterating stopped, epsilon-optimal policy found.
States:[0 0], Values:1450.3203405495542, Policy:0
States:[0 1], Values:1430.687889540296, Policy:2
States:[0 2], Values:1405.5890946374022, Policy:2
States:[0 3], Values:1375.0239994171407, Policy:2
States:[0 4], Values:1338.9932258531674, Policy:2
States:[0 5], Values

In [59]:
rvi = mdptoolbox.mdp.RelativeValueIteration(P2,R2)
rvi.setVerbose()

rvi.run()
result = rvi.policy # result is (0, 0, 0)import mdptoolbox.example
for i in range(len(states)):
    print(f"States:{states[i]}, Values:{rvi.V[i]}, Policy:{result[i]}")

  Iteration		U variation
    1		  426.0427870757492
    2		  342.9157702098371
    3		  261.7585483228851
    4		  180.5014945365885
    5		  99.86566618751112
    6		  47.19269806777902
    7		  20.63584803744095
    8		  8.80516187080758
    9		  3.894201119493175
    10		  1.8205058095664413
    11		  0.9181794524377551
    12		  0.4837262033216234
    13		  0.26499358561062536
    14		  0.14555570207511437
    15		  0.08072191342608903
    16		  0.04411927889839262
    17		  0.02413792794766323
    18		  0.012969398347649985
    19		  0.006971712867596125
Iterating stopped, epsilon-optimal policy found.
States:[0 0], Values:1387.394119779321, Policy:0
States:[0 1], Values:1383.492523152957, Policy:2
States:[0 2], Values:1374.1246246612286, Policy:2
States:[0 3], Values:1359.290418384827, Policy:2
States:[0 4], Values:1338.99070581555, Policy:2
States:[0 5], Values:1313.2303865653698, Policy:2
States:[1 0], Values:1383.492523152957, Policy:1
States:[1 1], Values:1374.1246246612286, 

In [60]:
import numpy as _np
global check, time_for_discounting

check = True
class ValueIterationUpdated(mdptoolbox.mdp.ValueIteration):

    def _bellmanOperator(self, V=None):
        global time_for_discounting, check
        if check:
            print("Custom implementation is being used")
            check = False
        # Apply the Bellman operator on the value function.
        #
        # Updates the value function and the Vprev-improving policy.
        #
        # Returns: (policy, value), tuple of new policy and its value
        #
        # If V hasn't been sent into the method, then we assume to be working
        # on the objects V attribute
        if V is None:
            # this V should be a reference to the data rather than a copy
            V = self.V
        else:
            # make sure the user supplied V is of the right shape
            try:
                assert V.shape in ((self.S,), (1, self.S)), "V is not the " \
                    "right shape (Bellman operator)."
            except AttributeError:
                raise TypeError("V must be a numpy array or matrix.")
        # Looping through each action the the Q-value matrix is calculated.
        # P and V can be any object that supports indexing, so it is important
        # that you know they define a valid MDP before calling the
        # _bellmanOperator method. Otherwise the results will be meaningless.
        Q = _np.empty((self.A, self.S))
        for aa in range(self.A):
            Q[aa] = self.R[aa] + np.exp(-time_for_discounting[aa]*self.discount) * self.P[aa].dot(V)
        # Get the policy and value, for now it is being returned but...
        # Which way is better?
        # 1. Return, (policy, value)
        return (Q.argmax(axis=0), Q.max(axis=0))
        # 2. update self.policy and self.V directly
        # self.V = Q.max(axis=1)
        # self.policy = Q.argmax(axis=1)

In [61]:
Q = 5
asd = ValueIteration(Q,3,3,0.5,0.3,0.5,35, 1,0.2)
states = np.array([[i, j] for i in range(Q + 1) for j in range(Q - i + 1)])
P = np.zeros((3,len(states),len(states)))
R = np.zeros((3,len(states),len(states)))
for actions in [0,1,2]:
    for i in range(len(states)):
        transition_set = asd.get_transition_set(states[i],actions)
        for next_state,prob,cost,arrival_time in transition_set:
            P[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = prob
            R[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = -cost

time_for_discounting = {
    0 : 1/asd.arrival_rate,
    1 : asd.Tp,
    2 : asd.Tnp
}

vi = ValueIterationUpdated(P1, R1,0.8)
vi.run()
result = vi.policy # result is (0, 0, 0)import mdptoolbox.example
for i in range(len(states)):
    print(f"States:{states[i]}, Values:{vi.V[i]}, Policy:{result[i]}")

check = True

Custom implementation is being used
States:[0 0], Values:-0.3350415357126446, Policy:0
States:[0 1], Values:-5.412926647988585, Policy:0
States:[0 2], Values:-10.490827463780933, Policy:0
States:[0 3], Values:-15.570408782979662, Policy:0
States:[0 4], Values:-20.829663058725583, Policy:0
States:[0 5], Values:-45.298096152401214, Policy:2
States:[1 0], Values:-31.172373689028277, Policy:1
States:[1 1], Values:-34.59924048823092, Policy:1
States:[1 2], Values:-38.02614315319525, Policy:1
States:[1 3], Values:-41.45688493823617, Policy:1
States:[1 4], Values:-45.298096152401214, Policy:1
States:[2 0], Values:-140.5433091437508, Policy:1
States:[2 1], Values:-143.85176473023557, Policy:1
States:[2 2], Values:-147.16030222683156, Policy:1
States:[2 3], Values:-150.47761018064432, Policy:1
States:[3 0], Values:-255.57871799311454, Policy:1
States:[3 1], Values:-258.8786817442317, Policy:1
States:[3 2], Values:-262.1788325491774, Policy:1
States:[4 0], Values:-371.02267793385573, Policy:1
St

In [62]:
Q = 5
asd = ValueIteration(Q,3,3,0.5,0.3,0.5,35, 1,0.2)
transition_reward_matrix = asd.get_transition_reward_matrix()

states = np.array([[i, j] for i in range(Q + 1) for j in range(Q - i + 1)])
P2 = np.zeros((3,len(states),len(states)))
R2 = np.zeros((3,len(states),len(states)))
for actions in [0,1,2]:
    for i in range(len(states)):
        transition_set = transition_reward_matrix[tuple(states[i])][actions]
        for next_state,prob,cost in transition_set:
            P2[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = prob
            R2[actions,i,(states[:,0] == next_state[0]) & (states[:,1] == next_state[1])] = -cost


time_for_discounting = {
    0 : 1/asd.arrival_rate,
    1 : asd.Tp,
    2 : asd.Tnp
}

vi = ValueIterationUpdated(P2, R2,0.8)
vi.run()
result = vi.policy # result is (0, 0, 0)import mdptoolbox.example
for i in range(len(states)):
    print(f"States:{states[i]}, Values:{vi.V[i]}, Policy:{result[i]}")

check = True

Custom implementation is being used
States:[0 0], Values:-0.10495008534279242, Policy:0
States:[0 1], Values:-5.177615645832089, Policy:0
States:[0 2], Values:-9.697491949956946, Policy:2
States:[0 3], Values:-13.06015965472841, Policy:2
States:[0 4], Values:-16.362753827236947, Policy:2
States:[0 5], Values:-19.662228908676756, Policy:2
States:[1 0], Values:-6.282549465259967, Policy:1
States:[1 1], Values:-9.697491949956946, Policy:1
States:[1 2], Values:-13.06015965472841, Policy:1
States:[1 3], Values:-16.362753827236947, Policy:1
States:[1 4], Values:-19.662228908676756, Policy:1
States:[2 0], Values:-113.8847415107394, Policy:1
States:[2 1], Values:-117.19124845342161, Policy:1
States:[2 2], Values:-120.49384262593016, Policy:1
States:[2 3], Values:-123.79331770736997, Policy:1
States:[3 0], Values:-228.79350433679812, Policy:1
States:[3 1], Values:-232.0932440357185, Policy:1
States:[3 2], Values:-235.39271911715826, Policy:1
States:[4 0], Values:-344.2284187363353, Policy:1
Sta